# Module 1.2: The ReAct Pattern - Complete Implementation 🔄

**Duration**: 30 minutes  
**Level**: Foundation  

## 🎯 Learning Objectives

By the end of this module, you'll understand:

1. **Agent Architecture**: The four core components every agent needs
2. **ReAct Pattern**: How agents think, act, and learn from observations  
3. **State Management**: Tracking agent progress and context
4. **Tool Integration**: Building and using external capabilities
5. **Function Calling**: Structured communication with language models

## 🚀 Prerequisites

Before starting, ensure you have:
```bash
# Ollama with Qwen2.5 7B Instruct
ollama pull qwen2.5:7b-instruct-q4_K_M
```

## 🔧 Environment Setup

In [1]:
# Configuration
MODEL_NAME = "qwen2.5:7b-instruct-q4_K_M"  # Qwen2.5 7B Instruct quantized model
OLLAMA_BASE_URL = "http://localhost:11434"

# Test the connection
import requests
import json
from enum import Enum
from dataclasses import dataclass, field
from typing import List, Dict, Any, Optional, Tuple
import time

try:
    # Test basic connectivity
    response = requests.get(f"{OLLAMA_BASE_URL}/api/tags", timeout=5)
    if response.status_code == 200:
        print("✅ Ollama server is running")
        
        # Test model availability
        models = response.json().get('models', [])
        model_names = [model['name'] for model in models]
        
        if MODEL_NAME in model_names:
            print(f"✅ {MODEL_NAME} is available")
        else:
            print(f"❌ {MODEL_NAME} not found. Available models: {model_names}")
            print("Run: ollama pull qwen2.5:7b-instruct-q4_K_M")
    else:
        print(f"❌ Ollama server responded with status {response.status_code}")
except requests.exceptions.RequestException as e:
    print(f"❌ Cannot connect to Ollama: {e}")
    print("Make sure Ollama is installed and running (ollama serve)")

## 🏗️ Part 1: Core Agent Architecture

We'll construct our agent using these fundamental components:

1. **State** - The agent's memory and context
2. **Brain** - LLM integration for reasoning
3. **Tools** - Capabilities beyond text generation
4. **Controller** - Orchestrates the agent loop

In [2]:
# First, let's define agent states - this helps us track what the agent is doing
class AgentState(Enum):
    """Possible states for our agent - like a state machine"""
    IDLE = "idle"              # Waiting for a task
    THINKING = "thinking"      # Processing with LLM
    ACTING = "acting"          # Executing a tool
    OBSERVING = "observing"    # Processing tool results
    COMPLETED = "completed"    # Task finished
    ERROR = "error"           # Something went wrong

# Define what an action looks like
@dataclass
class AgentAction:
    """
    Represents a single action the agent wants to take.
    This is the bridge between thinking and doing.
    """
    tool_name: str          # Which tool to use
    tool_input: str         # What to pass to the tool
    reasoning: str          # Why this action was chosen
    confidence: float = 0.0 # How confident (0-1)

# Configuration for our agent
@dataclass
class AgentConfig:
    """
    Configuration parameters for agent behavior.
    These knobs let us tune how the agent operates.
    """
    name: str = "Agent"
    model: str = "qwen2.5:7b-instruct-q4_K_M"  
    max_iterations: int = 5          # Prevent infinite loops
    verbose: bool = True             # Show reasoning process
    temperature: float = 0.7         # LLM creativity (0=deterministic, 1=creative)
    timeout_seconds: int = 30        # Max time per action
    
# The context/memory structure
@dataclass 
class AgentContext:
    """
    The agent's working memory - everything it needs to remember.
    This is crucial for maintaining context across actions.
    """
    goal: str                           # What we're trying to achieve
    conversation_history: List[Dict] = field(default_factory=list)  # Past interactions
    action_history: List[AgentAction] = field(default_factory=list) # What we've done
    observations: List[str] = field(default_factory=list)           # What we've learned
    current_plan: List[str] = field(default_factory=list)          # Steps to take
    iteration: int = 0                  # How many cycles we've done
    start_time: float = field(default_factory=time.time)           # When we started
    
    def add_to_history(self, role: str, content: str):
        """Add an interaction to conversation history"""
        self.conversation_history.append({
            "role": role,
            "content": content,
            "timestamp": time.time()
        })
    
    def get_summary(self) -> str:
        """Get a summary of current context for the LLM"""
        summary = f"Goal: {self.goal}\n"
        summary += f"Iteration: {self.iteration}\n"
        
        if self.action_history:
            summary += "\nPrevious actions:\n"
            for action in self.action_history[-3:]:  # Last 3 actions
                summary += f"- {action.tool_name}: {action.reasoning}\n"
        
        if self.observations:
            summary += "\nKey observations:\n"
            for obs in self.observations[-3:]:  # Last 3 observations
                summary += f"- {obs}\n"
                
        return summary

print("✅ Core data structures defined!")

## 🧠 Part 2: LLM Integration with Ollama

The LLM serves as the agent's "brain" - it:
- **Reasons** about the current situation
- **Decides** what action to take next
- **Interprets** results from tools
- **Plans** multi-step solutions

In [3]:
class OllamaLLM:
    """
    Our interface to Ollama - handles all LLM communication.
    This is our agent's 'brain' that does the reasoning.
    """
    
    def __init__(self, model: str = "qwen2.5:7b-instruct-q4_K_M", temperature: float = 0.7):
        self.model = model
        self.temperature = temperature
        self.base_url = "http://localhost:11434"
        
    def generate(self, prompt: str, system: str = "") -> str:
        """
        Generate a response from the LLM.
        
        Args:
            prompt: The user prompt
            system: System prompt to set behavior
            
        Returns:
            The LLM's response text
        """
        # Combine system and user prompts
        full_prompt = f"{system}\n\nUser: {prompt}\n\nAssistant:" if system else prompt
        
        try:
            response = requests.post(
                f"{self.base_url}/api/generate",
                json={
                    "model": self.model,
                    "prompt": full_prompt,
                    "temperature": self.temperature,
                    "stream": False
                },
                timeout=30
            )
            
            if response.status_code == 200:
                return response.json().get('response', '')
            else:
                raise Exception(f"Ollama error: {response.status_code}")
                
        except requests.exceptions.Timeout:
            return "Error: LLM request timed out. Try a shorter prompt."
        except Exception as e:
            return f"Error: {str(e)}"
    
    def generate_structured(self, prompt: str, system: str = "") -> Dict[str, Any]:
        """
        Generate a structured response (JSON) from the LLM.
        This is crucial for agent actions that need parsing.
        """
        # Add JSON instruction to prompt
        json_prompt = f"{prompt}\n\nRespond ONLY with valid JSON, no other text."
        
        response = self.generate(json_prompt, system)
        
        # Try to parse JSON from response
        try:
            # Clean up response - LLMs sometimes add extra text
            json_str = response.strip()
            if "```json" in json_str:
                json_str = json_str.split("```json")[1].split("```")[0]
            elif "```" in json_str:
                json_str = json_str.split("```")[1].split("```")[0]
            
            return json.loads(json_str)
        except:
            # Fallback for parsing errors
            return {
                "error": "Failed to parse LLM response as JSON",
                "raw_response": response
            }

# Test the LLM integration
llm = OllamaLLM(model=MODEL_NAME)

print("🧪 Testing LLM integration...")
test_response = llm.generate(
    "Hello! Please respond with: 'LLM integration successful'",
    system="You are a helpful assistant."
)
print(f"📝 LLM Response: {test_response[:100]}...")

## 🔧 Part 3: Building Tools

Tools extend the agent's capabilities beyond text generation. They're the agent's way of:
- **Accessing** external information
- **Performing** calculations
- **Interacting** with systems
- **Storing** and retrieving data

In [4]:
# Base tool interface
class Tool:
    """
    Base class for all tools. Every tool must:
    1. Have a name and description
    2. Implement the execute method
    3. Handle errors gracefully
    """
    
    def __init__(self, name: str, description: str):
        self.name = name
        self.description = description
        
    def execute(self, input_str: str) -> str:
        """Execute the tool with given input"""
        raise NotImplementedError("Subclasses must implement execute")
        
    def validate_input(self, input_str: str) -> Tuple[bool, str]:
        """Validate input before execution"""
        if not input_str or not isinstance(input_str, str):
            return False, "Input must be a non-empty string"
        return True, "Valid"

In [5]:
class SearchTool(Tool):
    """
    Simulated web search tool.
    In production, this would call a real search API.
    """
    
    def __init__(self):
        super().__init__(
            name="search",
            description="Search for information on any topic. Input: search query"
        )
        # Simulated knowledge base
        self.knowledge_base = {
            "ai agents": "AI agents are autonomous systems that perceive, reason, and act to achieve goals.",
            "react pattern": "ReAct combines reasoning and acting in a loop for better agent behavior.",
            "llm": "Large Language Models are neural networks trained on vast text data.",
            "python": "Python is a high-level programming language known for simplicity.",
            "climate": "Climate change refers to long-term shifts in global temperatures."
        }
    
    def execute(self, query: str) -> str:
        valid, msg = self.validate_input(query)
        if not valid:
            return f"Search error: {msg}"
            
        query_lower = query.lower()
        
        # Find relevant results
        results = []
        for key, value in self.knowledge_base.items():
            if any(word in query_lower for word in key.split()):
                results.append(value)
        
        if results:
            return f"Search results for '{query}': " + " ".join(results[:2])
        else:
            return f"No specific results found for '{query}'. Try different keywords."

# Test the search tool
search_tool = SearchTool()
print(f"Search test: {search_tool.execute('AI agents')}")

In [6]:
class CalculatorTool(Tool):
    """
    Safe calculator for mathematical expressions.
    Uses eval() with strict input validation.
    """
    
    def __init__(self):
        super().__init__(
            name="calculator", 
            description="Perform mathematical calculations. Input: mathematical expression"
        )
        self.allowed_chars = set('0123456789+-*/()., ')
        self.allowed_names = {'abs', 'round', 'min', 'max'}
    
    def execute(self, expression: str) -> str:
        valid, msg = self.validate_input(expression)
        if not valid:
            return f"Calculator error: {msg}"
        
        # Security: validate expression characters
        if not all(c in self.allowed_chars for c in expression):
            return "Error: Invalid characters in expression. Use only numbers and +-*/()."
        
        try:
            # Create safe namespace
            safe_dict = {name: getattr(__builtins__, name) for name in self.allowed_names}
            result = eval(expression, {"__builtins__": {}}, safe_dict)
            return f"Result: {result}"
        except ZeroDivisionError:
            return "Error: Division by zero"
        except Exception as e:
            return f"Error: Invalid expression - {str(e)}"

# Test calculator
calc_tool = CalculatorTool()
print(f"Calculator test: {calc_tool.execute('(10 + 5) * 2')}")

In [7]:
class MemoryTool(Tool):
    """
    Simple key-value memory storage.
    Allows agent to store and retrieve information.
    """
    
    def __init__(self):
        super().__init__(
            name="memory",
            description="Store or retrieve information. Input: 'store key=value' or 'get key'"
        )
        self.storage = {}
    
    def execute(self, command: str) -> str:
        valid, msg = self.validate_input(command)
        if not valid:
            return f"Memory error: {msg}"
        
        # First check if it starts with 'store ' or 'get '
        if command.startswith('store '):
            # Extract the key=value part
            data = command[6:].strip()  # Remove 'store '
            if '=' not in data:
                return "Error: Store format is 'store key=value'"
            key, value = data.split('=', 1)
            self.storage[key.strip()] = value.strip()
            return f"Stored: {key.strip()} = {value.strip()}"
            
        elif command.startswith('get '):
            # Extract the key
            key = command[4:].strip()  # Remove 'get '
            if key in self.storage:
                return f"Retrieved: {key} = {self.storage[key]}"
            else:
                return f"Not found: {key}"
                
        else:
            return "Error: Command must start with 'store ' or 'get '"

# Create tool collection
tools = {
    "search": SearchTool(),
    "calculator": CalculatorTool(), 
    "memory": MemoryTool()
}

print("🔧 Tool collection created:")
for name, tool in tools.items():
    print(f"  - {name}: {tool.description}")

## 🔄 Part 4: The ReAct Pattern

### What is ReAct?

ReAct (Reasoning + Acting) is a cognitive architecture that interleaves:
- **Reasoning**: Thinking about what to do
- **Acting**: Actually doing it
- **Observing**: Learning from results

### Why ReAct Works

Traditional approaches separate thinking and acting. ReAct combines them:

```
Traditional:             ReAct:
Think → Think → Act      Think → Act → Observe → Think → Act → Observe
        ↓                                                         ↓
   Often wrong                                        Self-correcting
```

In [8]:
class ReActAgent:
    """
    Our main agent class implementing the ReAct pattern.
    This is where everything comes together!
    """
    
    def __init__(self, config: AgentConfig):
        self.config = config
        self.llm = OllamaLLM(model=config.model, temperature=config.temperature)
        self.tools = {}
        self.state = AgentState.IDLE
        self.context = None
        
    def add_tool(self, tool: Tool):
        """Register a tool with the agent"""
        self.tools[tool.name] = tool
        if self.config.verbose:
            print(f"✅ Added tool: {tool.name}")
    
    def _update_state(self, new_state: AgentState):
        """Update agent state with logging"""
        if self.config.verbose:
            print(f"\n🔄 State: {self.state.value} → {new_state.value}")
        self.state = new_state
    
    def _create_system_prompt(self) -> str:
        """Create the system prompt that defines agent behavior"""
        tool_descriptions = "\n".join([
            f"- {name}: {tool.description}"
            for name, tool in self.tools.items()
        ])
        
        return f"""You are {self.config.name}, an autonomous AI agent using the ReAct pattern.

You have access to these tools:
{tool_descriptions}

For each step, you must:
1. THOUGHT: Analyze the current situation and plan your next action
2. ACTION: Choose a tool and provide input
3. Wait for OBSERVATION
4. Repeat until the goal is achieved

IMPORTANT: 
- Always start with a THOUGHT
- Use tools to gather information or perform actions
- Be concise and focused
- Learn from observations to improve your approach

Format your response as:
THOUGHT: [your reasoning]
ACTION: [tool_name] [input]
"""
    
    def _parse_llm_response(self, response: str) -> Optional[AgentAction]:
        """Parse LLM response to extract action"""
        lines = response.strip().split('\n')
        
        thought = ""
        action_line = ""
        
        for line in lines:
            if line.strip().startswith("THOUGHT:"):
                thought = line.replace("THOUGHT:", "").strip()
            elif line.strip().startswith("ACTION:"):
                action_line = line.replace("ACTION:", "").strip()
        
        if not action_line:
            return None
            
        # Parse action line
        parts = action_line.split(maxsplit=1)
        if len(parts) < 2:
            return None
            
        tool_name = parts[0]
        tool_input = parts[1] if len(parts) > 1 else ""
        
        return AgentAction(
            tool_name=tool_name,
            tool_input=tool_input,
            reasoning=thought
        )
    
    def _execute_action(self, action: AgentAction) -> str:
        """Execute an action using the appropriate tool"""
        if action.tool_name not in self.tools:
            return f"Error: Unknown tool '{action.tool_name}'"
            
        tool = self.tools[action.tool_name]
        
        try:
            result = tool.execute(action.tool_input)
            return result
        except Exception as e:
            return f"Error executing {action.tool_name}: {str(e)}"
    
    def think(self, context: AgentContext) -> Optional[AgentAction]:
        """Generate next action using LLM reasoning"""
        self._update_state(AgentState.THINKING)
        
        # Build prompt with context
        prompt = f"""Current context:
{context.get_summary()}

What should I do next to achieve the goal?
"""
        
        # Get LLM response
        response = self.llm.generate(prompt, self._create_system_prompt())
        
        if self.config.verbose:
            print(f"\n💭 LLM Response:\n{response}")
        
        # Parse action from response
        action = self._parse_llm_response(response)
        
        if action:
            context.add_to_history("assistant", response)
            context.action_history.append(action)
            
        return action
    
    def act(self, action: AgentAction) -> str:
        """Execute the chosen action"""
        self._update_state(AgentState.ACTING)
        
        if self.config.verbose:
            print(f"\n🔧 Executing: {action.tool_name} with input: {action.tool_input}")
        
        result = self._execute_action(action)
        
        return result
    
    def observe(self, observation: str, context: AgentContext):
        """Process the observation from action"""
        self._update_state(AgentState.OBSERVING)
        
        if self.config.verbose:
            print(f"\n👁️ Observation: {observation}")
        
        context.observations.append(observation)
        context.add_to_history("observation", observation)
    
    def run(self, goal: str) -> str:
        """Run the agent to achieve a goal"""
        print(f"\n🎯 Starting agent with goal: {goal}")
        
        # Initialize context
        context = AgentContext(goal=goal)
        self.context = context
        
        # Main agent loop
        while context.iteration < self.config.max_iterations:
            context.iteration += 1
            
            if self.config.verbose:
                print(f"\n{'='*50}")
                print(f"Iteration {context.iteration}/{self.config.max_iterations}")
                print(f"{'='*50}")
            
            # Think
            action = self.think(context)
            if not action:
                print("\n❌ Could not determine next action")
                break
            
            # Act
            result = self.act(action)
            
            # Observe
            self.observe(result, context)
            
            # Check if goal achieved (simple heuristic)
            if "error" not in result.lower() and context.iteration > 1:
                # Ask LLM if goal is achieved
                check_prompt = f"""Based on the context and observations, has the goal been achieved?
Goal: {goal}
Latest observation: {result}

Answer with just YES or NO."""
                
                check_response = self.llm.generate(check_prompt).strip().upper()
                if "YES" in check_response:
                    self._update_state(AgentState.COMPLETED)
                    print("\n✅ Goal achieved!")
                    break
        
        # Prepare final summary
        if context.iteration >= self.config.max_iterations:
            print("\n⏰ Reached maximum iterations")
        
        return self._generate_summary(context)
    
    def _generate_summary(self, context: AgentContext) -> str:
        """Generate a summary of the agent's work"""
        summary = f"\n📊 Agent Summary:\n"
        summary += f"Goal: {context.goal}\n"
        summary += f"Iterations: {context.iteration}\n"
        summary += f"Actions taken: {len(context.action_history)}\n"
        summary += f"Final state: {self.state.value}\n"
        
        if context.observations:
            summary += f"\nKey findings:\n"
            for obs in context.observations[-3:]:
                summary += f"- {obs}\n"
        
        return summary

# Create and configure agent
print("\n🤖 Creating ReAct agent...")
agent_config = AgentConfig(
    name="ResearchBot",
    model=MODEL_NAME,
    max_iterations=5,
    verbose=True
)

agent = ReActAgent(agent_config)

# Add tools
for tool in tools.values():
    agent.add_tool(tool)

print("\n✅ Agent ready!")

## 🚀 Part 5: Agent in Action

Let's see our agent solve real problems. Watch how it:
1. Breaks down the goal
2. Chooses appropriate tools
3. Learns from observations
4. Achieves the objective

In [9]:
# Demo 1: Simple calculation task
print("📝 Demo 1: Simple Calculation")
print("Task: Calculate the area of a rectangle with width 15 and height 23")
print("-" * 60)

result = agent.run("Calculate the area of a rectangle with width 15 and height 23")
print(result)

In [10]:
# Demo 2: Multi-step research task
print("\n📝 Demo 2: Research Task")
print("Task: Research AI agents and store key findings")
print("-" * 60)

result = agent.run("Research what AI agents are and store the key findings in memory")
print(result)

## 🎯 Hands-On Exercise: Build Your Own Tool

Create a custom tool that the agent can use:

In [ ]:
# Exercise: Build a Weather Tool
class WeatherTool(Tool):
    """
    TODO: Implement a weather tool that:
    1. Takes a city name as input
    2. Returns mock weather data
    3. Handles invalid cities gracefully
    """
    
    def __init__(self):
        super().__init__(
            name="weather",
            description="Get weather for a city. Input: city name"
        )
        # TODO: Add mock weather data
        self.weather_data = {
            "london": "Cloudy, 15°C, 70% humidity",
            "paris": "Sunny, 22°C, 50% humidity",
            "tokyo": "Rainy, 18°C, 80% humidity",
            "new york": "Clear, 20°C, 60% humidity"
        }
    
    def execute(self, city: str) -> str:
        # TODO: Implement weather lookup
        valid, msg = self.validate_input(city)
        if not valid:
            return f"Weather error: {msg}"
            
        city_lower = city.lower().strip()
        
        if city_lower in self.weather_data:
            return f"Weather in {city}: {self.weather_data[city_lower]}"
        else:
            return f"Weather data not available for {city}"

# Test your implementation
weather_tool = WeatherTool()
print(weather_tool.execute("London"))
print(weather_tool.execute("Mumbai"))

# Add to agent
agent.add_tool(weather_tool)

# Test with agent
result = agent.run("What's the weather in Paris and Tokyo?")
print(result)

## 📊 Key Takeaways

### What You've Learned:

1. **Agent Architecture**: State, Context, Actions, and Control Flow
2. **ReAct Pattern**: Think → Act → Observe loop enables self-correction
3. **Tool Design**: Extend capabilities with clean interfaces
4. **LLM Integration**: Structure prompts for reliable agent behavior
5. **Error Handling**: Make agents robust with validation and fallbacks

### ReAct Performance:

From the research:
- **34% improvement** on ALFWorld tasks
- **10% improvement** on WebShop navigation
- Key: Interleaving reasoning with action

### Next Steps:

In Module 1.3, we'll explore:
- **ReWOO**: 64% token reduction vs ReAct
- Side-by-side performance comparison
- When to use each pattern

Ready to optimize? Let's go! 🚀